In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple

# Constants
C1 = 2.0
C2 = 2.0
W = 0.7
MAX_ITER =  300
POPULATION_SIZE = 100


COST_COEFFS = {
    'Diesel': (14.88, 0.300, 0.000435),
    'NaturalGas': (9.00, 0.306, 0.000315),
}


POWER_LIMITS = {
    'Diesel': (0, 1),
    'NaturalGas': (0, 1),
    'Solar': (0, 0.5),
    'Wind': (0, 0.5),
    'ESS': (0.1, 0.4),
    'Grid': (-1, 1)  # Negative for grid supply, positive for grid absorption
}


def generator_cost(power: float, gen_type: str) -> float:
    a, b, c = COST_COEFFS[gen_type]
    return a + b * power + c * power**2

def solar_cost(power: float) -> float:
    return 545.016 * power

def wind_cost(power: float) -> float:
    return 152.616 * power

def ess_cost(power: float) -> float:
    return 119 * power


def total_cost(powers: List[float], solar_power: float, wind_power: float, grid_power: float = 0.0) -> float:
    cost = (
        generator_cost(powers[0], 'Diesel') +
        generator_cost(powers[1], 'NaturalGas') +
        solar_cost(solar_power) +
        wind_cost(wind_power) +
        ess_cost(powers[2])
    )

    GRID_COST_COEFF = 2.0
    cost += GRID_COST_COEFF * abs(grid_power)
    return cost


def pso_optimize(load_demand: float, solar_power: float, wind_power: float, grid_connected: bool = False) -> Tuple[List[float], float, float]:
    """
    Returns:
        best_powers: List containing [Diesel, NaturalGas, ESS]
        best_fitness: Total cost
        grid_power: Power exchanged with the grid (only if grid_connected is True)
    """

    GRID_COST_COEFF = 2.0


    def fitness(particle):
        diesel, nat_gas, ess = particle
        total_generation = diesel + nat_gas + ess + solar_power + wind_power
        grid_power = total_generation-load_demand if grid_connected else 0.0
        grid_power = np.clip(grid_power, POWER_LIMITS['Grid'][0], POWER_LIMITS['Grid'][1]) if grid_connected else 0.0
        penalty = 0 if grid_connected else abs(total_generation - load_demand)

        cost = total_cost(particle, solar_power, wind_power, grid_power) + penalty
        return cost


    diesel_min, diesel_max = POWER_LIMITS['Diesel']
    nat_gas_min, nat_gas_max = POWER_LIMITS['NaturalGas']
    ess_min, ess_max = POWER_LIMITS['ESS']

    particles = np.random.uniform(
        low=[diesel_min, nat_gas_min, ess_min],
        high=[diesel_max, nat_gas_max, ess_max],
        size=(POPULATION_SIZE, 3)
    )
    velocities = np.zeros_like(particles)
    pbest = particles.copy()
    pbest_fitness = np.array([fitness(p) for p in pbest])
    gbest = pbest[pbest_fitness.argmin()].copy()
    gbest_fitness = pbest_fitness.min()

    for _ in range(MAX_ITER):
        for i in range(POPULATION_SIZE):
            r1 = np.random.rand(3)
            r2 = np.random.rand(3)
            velocities[i] = (
                W * velocities[i] +
                C1 * r1 * (pbest[i] - particles[i]) +
                C2 * r2 * (gbest - particles[i])
            )
            particles[i] += velocities[i]

            particles[i] = np.clip(
                particles[i],
                [diesel_min, nat_gas_min, ess_min],
                [diesel_max, nat_gas_max, ess_max]
            )
            current_fitness = fitness(particles[i])
            if current_fitness < pbest_fitness[i]:
                pbest[i] = particles[i].copy()
                pbest_fitness[i] = current_fitness
                if current_fitness < gbest_fitness:
                    gbest = particles[i].copy()
                    gbest_fitness = current_fitness


        diesel, nat_gas, ess = gbest
    total_generation = diesel + nat_gas + ess + solar_power + wind_power
    grid_power = load_demand - total_generation if grid_connected else 0.0
    grid_power = np.clip(grid_power, POWER_LIMITS['Grid'][0], POWER_LIMITS['Grid'][1]) if grid_connected else 0.0

    return gbest.tolist(), gbest_fitness, grid_power

def generate_random_profile(min_val: float, max_val: float, length: int) -> np.ndarray:
    return np.random.uniform(min_val, max_val, length)


hours = np.arange(1, 25)
load_demand_grid=np.array([1.5,1.5,2.5,1.6,2.4,1.7,1.9,1.4,1.5,2.3,1.5,1.4,1.5,2.3,1.3,1.7,1.6,1.3,1.2,1.3,1.6,1.4,1.6,1.3])
# load_demand_grid=np.array([3,3.25,3.5,4,4.25,4.25,4.125,3.75,3.15,3.25,3.15,3.15,3.15,3.25,3.5,3.75,4,4.25,4.25,4,3.8,3.5,3.10,3.15])
load_demand_islanded=np.array([2.1,2.1,2.15,2.2,2.25,2.25,2.25,2.3,2.3,2.35,2.4,2.4,2.4,2.3,2.3,2.2,2.2,2.1,2.1,2.05,2.05,2,2,1.9])
solar_power = generate_random_profile(0, POWER_LIMITS['Solar'][1], 24)
wind_power = generate_random_profile(0, POWER_LIMITS['Wind'][1], 24)


grid_connected_results = [pso_optimize(load, solar, wind, grid_connected=True) for load, solar, wind in zip(load_demand_grid, solar_power, wind_power)]
islanded_results = [pso_optimize(load, solar, wind, grid_connected=False) for load, solar, wind in zip(load_demand_islanded, solar_power, wind_power)]


grid_connected_powers = np.array([result[0] for result in grid_connected_results])
grid_connected_costs = np.array([result[1] for result in grid_connected_results])
grid_powers = np.array([result[2] for result in grid_connected_results])

islanded_powers = np.array([result[0] for result in islanded_results])
islanded_costs = np.array([result[1] for result in islanded_results])
load_demand_grid1=np.array([1.5,1.5,2.5,1.6,2.4,1.7,1.9,1.4,1.5,2.3,1.5,1.4,1.5,2.3,1.3,1.7,1.6,1.3,1.2,1.3,1.6,1.4,1.6,1.3])

plt.figure(figsize=(14, 10))

# Grid-Connected Mode Plots
plt.subplot(2, 1, 1)
plt.plot(hours, grid_connected_powers[:, 0], 'b--', label='Diesel (Gen1)', linewidth=2)
plt.plot(hours, grid_connected_powers[:, 1], 'g-', label='Natural Gas (Gen2)', linewidth=2)
plt.plot(hours, grid_connected_powers[:, 2], 'c--', label='ESS', linewidth=2)
plt.plot(hours, solar_power, color='black', label='Solar', linewidth=2)
plt.plot(hours, wind_power, color='orange', label='Wind', linewidth=2)
plt.plot(hours, load_demand_grid, 'r--', label='Load Demand', linewidth=2)
plt.plot(hours, grid_powers, 'm-.', label='Grid Power', linewidth=2)

plt.grid(True, linestyle='--', alpha=0.7)
plt.xlabel('Time (h)')
plt.ylabel('Power (MW)')
plt.title('Grid-Connected Mode')
plt.legend(loc='upper left')
plt.xlim(1, 24)
plt.xticks(np.arange(1, 25, 1))
plt.ylim(-1.5, 6)
plt.yticks(np.arange(-1.5, 6.5, 0.5))

# Islanded Mode Plots
plt.subplot(2, 1, 2)
plt.plot(hours, islanded_powers[:, 0], 'b--', label='Diesel (Gen1)', linewidth=2)
plt.plot(hours, islanded_powers[:, 1], 'g-', label='Natural Gas (Gen2)', linewidth=2)
plt.plot(hours, islanded_powers[:, 2], 'c--', label='ESS', linewidth=2)
plt.plot(hours, solar_power, color='black', label='Solar', linewidth=2)
plt.plot(hours, wind_power, color='orange', label='Wind', linewidth=2)
plt.plot(hours, load_demand_islanded, 'r--', label='Load Demand', linewidth=2)

plt.grid(True, linestyle='--', alpha=0.7)
plt.xlabel('Time (h)')
plt.ylabel('Power (MW)')
plt.title('Islanded Mode')
plt.legend(loc='upper left')
plt.xlim(1, 24)
plt.xticks(np.arange(1, 25, 1))
plt.ylim(0, 3.5)
plt.yticks(np.arange(0, 3.6, 0.5))

plt.tight_layout()
plt.show()


total_grid_connected_cost = grid_connected_costs.sum()
total_islanded_cost = islanded_costs.sum()

print(f"Total cost for grid-connected mode: ${total_grid_connected_cost:.2f}")
print(f"Total cost for islanded mode: ${total_islanded_cost:.2f}")

In [ ]:
for result in grid_connected_costs:
      # print(f"Hour {result['hour']}: Total Cost = ${result['cost']:.2f}, Cost/Hour = ${result['cost_per_hour']:.2f}")
      print(result)

for demand in load_demand_grid:
   print(demand)

144.27457194223751
212.06646767623016
321.7965636149296
84.15063612352405
249.32455750353907
85.65348088927935
85.92782587009958
309.78206452422774
297.11379608754356
203.5245831297501
233.37491634066308
175.38645547898352
147.39803896573594
210.30410871120958
108.53639131780932
208.1014730924796
233.59716449208955
131.26243445206475
293.45988925117376
220.46355654480269
120.18603898121225
217.943989830826
114.36243227818242
332.52071781650227
3.0
3.25
3.5
4.0
4.25
4.25
4.125
3.75
3.15
3.25
3.15
3.15
3.15
3.25
3.5
3.75
4.0
4.25
4.25
4.0
3.8
3.5
3.1
3.15


In [ ]:
print(f"Total cost for grid-connected mode: ${grid_connected_costs.sum():.2f}")
print(f"Total cost for islanded mode: ${islanded_costs.sum():.2f}")

Total cost for grid-connected mode: $4483.45
Total cost for islanded mode: $4458.41
